# Data Prep

The data source is a REDCap audit logging file, a REDCap report, and a CSV extract from Clarity. 

## Imports

In [1]:
import os 
import glob
import shutil 
import pandas as pd
from datetime import datetime
from datetime import date
from dateutil.relativedelta import *
import numpy as np

pd.set_option('display.max_colwidth', 75)
pd.set_option('display.max_rows', 200)


# import most recent logging file from REDCap (REDCap > Sidebar > Logging > Export all logging (CSV))
latest_log_filepath = max(glob.iglob('data/raw/UCI REDCap Log/*.csv'), key=os.path.getmtime)
log = pd.read_csv(latest_log_filepath)

# import most recent extract aka "report" of useful data from REDCap (REDCap > Sidebar > Data Exports, Reports, and Stats)
latest_redcap_extract_filepath = max(glob.iglob('data/raw/UCI REDCap Report/*.csv'), key=os.path.getmtime)
redcap_extract = pd.read_csv(latest_redcap_extract_filepath)

# copy to local storage most recent extract used for intervention tracking from Clarity 
latest_clarity_extract_filepath = max(glob.iglob('Z:/PCORI ACP Lists/*.csv'), key=os.path.getmtime)
copy_clarity_extract_filepath = ('data/raw/Clarity Extract/')
shutil.copy(latest_clarity_extract_filepath, copy_clarity_extract_filepath)

# import copy of most recent extract used for intervention tracking from Clarity
latest_copy_clarity_extract_filepath = max(glob.iglob('data/raw/Clarity Extract/*.csv'), key=os.path.getmtime)
clarity_extract = pd.read_csv(latest_copy_clarity_extract_filepath, encoding='latin-1')

# import copy of extract that was pulled 1 week after the end of the intervention (Orange Dot + 24 months)
static_24_month_clarity_extract = pd.read_csv('data/raw/Static Clarity Extract/20220419_PCORIWeeklyExtract.csv', encoding='latin-1')

# import mailing lists for 2nd round, duplicate baseline research surveys
round_2_mailing_a = pd.read_csv('data/raw/Baseline Survey Round 2 Mailing Lists/2.5.20_mailing_list_A.csv')
round_2_mailing_b = pd.read_csv('data/raw/Baseline Survey Round 2 Mailing Lists/2.5.20_mailing_list_B.csv')
round_2_mailing_c = pd.read_csv('data/raw/Baseline Survey Round 2 Mailing Lists/2.5.20_mailing_list_C.csv')
round_2_mailing_d = pd.read_csv('data/raw/Baseline Survey Round 2 Mailing Lists/2.5.20_mailing_list_D.csv')

# import most recent list of pop cohort study ID's with geocoded data (lat & long coordinates, Census Tract codes)
latest_pop_geoID_filepath = max(glob.iglob("C:/Users/chauaj1/OneDrive - UCI Health/Documents/Python Projects/Geocoding/data/processed/4-final_output/*.csv"), key=os.path.getmtime)
pop_geoID = pd.read_csv(latest_pop_geoID_filepath)

# import CDC's 2018 CSV file of California's Census Tracts + SVI 
# https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html
SVI_source = pd.read_csv('data/raw/CDC 2018 SVI/California.csv')

## UCI ONLY: Removable Variables

In [2]:
UCI_ONLY_OLD_ID = "..."
UCI_ONLY_NEW_ID = "..."
UCI_ONLY_OPTED_OUT_ID = "..."

UCI_ONLY_RES_EXC_DIED = ["..."]
UCI_ONLY_RES_EXC_ACP = ["..."]
UCI_ONLY_RES_EXC_LATE_SURVEY = ["..."]
UCI_ONLY_RES_EXC_LATE_RECEIPT = ["..."]

UCI_ONLY_GREY_DOT_EXC_PCP_NO_REASON = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_NO_ENG_SP = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_AD_POLST = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_DEMENTIA = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_WRONG_PCP = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_MOVED = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_HOSPICE = ["..."]
UCI_ONLY_GREY_DOT_EXC_PCP_INCAPABLE = ["..."]

UCI_ONLY_ORANGE_DOT_EXC_DECEASED = ["..."]
UCI_ONLY_ORANGE_DOT_EXC_HOSPICE = ["..."]

UCI_ONLY_MISSING_PATIENT_DATA = pd.DataFrame.from_dict({'study_id':[...],
                                               'birth_date':["..."],
                                               'sex':["..."],
                                               'ethnicity':["..."],
                                               'race':["..."],
                                               #'insurance':[], not available at the time of the old extract
                                               #'religion':[], not available at the time of the old extract
                                               'spoken_language':["..."],
                                               'mct_activation':["..."],
                                               'mct_status':["..."]})

## REDCap Log: Split into round 1 / 2 / 3 call dataframes

In [3]:
# make a new df with only REDCap Patient Record Updates
log_updates = log[log['Action'].str.contains("Updated")].copy()

# make a new df with only REDCap Patient Record Updates + Phone Call 1 / 2 / 3 Attempts
# assumes that RA updating the contact date field in Internal REDCAp = call completed
log_updates_call1 = log_updates[log_updates['List of Data Changes OR Fields Exported'].str.contains("contact1_dt ", na=False)].copy()
log_updates_call2 = log_updates[log_updates['List of Data Changes OR Fields Exported'].str.contains("contact2_dt ", na=False)].copy()
log_updates_call3 = log_updates[log_updates['List of Data Changes OR Fields Exported'].str.contains("contact3_dt ", na=False)].copy()

print("# of Round 1 Call Updates: ", log_updates_call1.shape[0])
print("# of Round 2 Call Updates: ", log_updates_call2.shape[0])
print("# of Round 3 Call Updates: ", log_updates_call3.shape[0])
print("\nTotal Call Updates: ", log_updates_call1.shape[0] + log_updates_call2.shape[0] + log_updates_call3.shape[0])

# of Round 1 Call Updates:  790
# of Round 2 Call Updates:  509
# of Round 3 Call Updates:  287

Total Call Updates:  1586


## REDCap Log: obtain study ID

In [4]:
# extract study ID from long string ('Action')
log_updates_call1['study_id'] = log_updates_call1['Action'].str.split(' ').str[2]
log_updates_call2['study_id'] = log_updates_call2['Action'].str.split(' ').str[2]
log_updates_call3['study_id'] = log_updates_call3['Action'].str.split(' ').str[2]

## REDCap Log: handle duplicates

In [5]:
# convert dtype from string to datetime
log_updates_call1['Time / Date'] = pd.to_datetime(log_updates_call1['Time / Date'])
log_updates_call2['Time / Date'] = pd.to_datetime(log_updates_call2['Time / Date'])
log_updates_call3['Time / Date'] = pd.to_datetime(log_updates_call3['Time / Date'])

# sort calls by date from oldest to newest
log_updates_call1 = log_updates_call1.sort_values(by=['Time / Date'])
log_updates_call2 = log_updates_call2.sort_values(by=['Time / Date'])
log_updates_call3 = log_updates_call3.sort_values(by=['Time / Date'])

# check for duplicate updates to 1 study ID
print("original round 1 duplicates: ", log_updates_call1[log_updates_call1['study_id'].duplicated(keep=False)].shape[0])
print("original round 2 duplicates: ", log_updates_call2[log_updates_call2['study_id'].duplicated(keep=False)].shape[0])
print("original round 3 duplicates: ", log_updates_call3[log_updates_call3['study_id'].duplicated(keep=False)].shape[0])

# when there are multiple updates to 1 study ID, only take the most recent update
unique_log_updates_call1 = log_updates_call1.drop_duplicates(subset=['study_id'], keep='last')
unique_log_updates_call2 = log_updates_call2.drop_duplicates(subset=['study_id'], keep='last')
unique_log_updates_call3 = log_updates_call3.drop_duplicates(subset=['study_id'], keep='last')

# check for duplicate updates to 1 study ID
print("\nnew round 1 duplicates: ", unique_log_updates_call1[unique_log_updates_call1['study_id'].duplicated(keep=False)].shape[0])
print("new round 2 duplicates: ", unique_log_updates_call2[unique_log_updates_call2['study_id'].duplicated(keep=False)].shape[0])
print("new round 3 duplicates: ", unique_log_updates_call3[unique_log_updates_call3['study_id'].duplicated(keep=False)].shape[0])

print("\n# of Round 1 Calls: ", unique_log_updates_call1.shape[0])
print("# of Round 2 Calls: ", unique_log_updates_call2.shape[0])
print("# of Round 3 Calls: ", unique_log_updates_call3.shape[0])
print("\nTotal Calls: ", unique_log_updates_call1.shape[0] + unique_log_updates_call2.shape[0] + unique_log_updates_call3.shape[0])

original round 1 duplicates:  0
original round 2 duplicates:  7
original round 3 duplicates:  2

new round 1 duplicates:  0
new round 2 duplicates:  0
new round 3 duplicates:  0

# of Round 1 Calls:  790
# of Round 2 Calls:  505
# of Round 3 Calls:  286

Total Calls:  1581


## REDCap Log: extract useful data from strings

In [6]:
# extract individual updates from long string ('List of Data Changes OR Fields Exported'), based on delimiter ','
split_unique_log_updates_call1 = pd.concat([unique_log_updates_call1['study_id'], 
                                     unique_log_updates_call1['List of Data Changes OR Fields Exported'].str.split(',', expand=True)],
                                    axis=1,)
split_unique_log_updates_call2 = pd.concat([unique_log_updates_call2['study_id'],
                                     unique_log_updates_call2['List of Data Changes OR Fields Exported'].str.split(',', expand=True)],
                                    axis=1,)
split_unique_log_updates_call3 = pd.concat([unique_log_updates_call3['study_id'],
                                     unique_log_updates_call3['List of Data Changes OR Fields Exported'].str.split(',', expand=True)],
                                    axis=1,)



# extract updates specific to round 1 / 2 / 3 phone calls 
# make a boolean mask of all columns in the round 1 / 2 / 3 dataframe, True for matching strings
# forward fill rows with matching strings and take only the last value 
# basically picks out the matching value regardless of column location and places it into the correct column

# round 1
df = split_unique_log_updates_call1.copy()
strings = ['contact1_dt', 'contact1_output', 'contact1_nt', 'verbal_yn']
updates_round_1 = pd.DataFrame()

for s in strings:
    for col in df: 
        df[col] = df[col].mask(~df[col].str.contains(s, na=False))
    updates_round_1[s] = df.ffill(axis=1).iloc[:, -1]
    df = split_unique_log_updates_call1.copy()

# extracts date from 'contact1_dt' output
updates_round_1.iloc[:,0] = pd.to_datetime(updates_round_1.iloc[:,0].str.extract('(\d{1,4}-\d{1,2}-\d{1,2})')[0]).dt.date

# convert call code outputs to real words, per UCI's REDCap Codebook 
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '1'": "No answer/unable to leave VM/busy/disconnected"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '2'": "Left a message"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '3'": "Call back later"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '4'": "Hasn't received packet yet, call back in one week"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '5'": "Hasn't received packet and team needs to resend in the mail"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '6'": "Send link to the survey"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '7'": "Completed survey by phone"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '8'": "Patient refused"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '9'": "Deceased"}, regex=True)
updates_round_1.iloc[:,1] = updates_round_1.iloc[:,1].replace({" contact1_output = '10'": "Other Notes"}, regex=True)

updates_round_1.iloc[:,3] = updates_round_1.iloc[:,3].replace({" verbal_yn = '0'": "No"}, regex=True)
updates_round_1.iloc[:,3] = updates_round_1.iloc[:,3].replace({" verbal_yn = '1'": "Yes"}, regex=True)
    
    
# round 2
df = split_unique_log_updates_call2.copy()
strings = ['contact2_dt', 'contact2_output', 'contact2_nt', 'verbal_yn']
updates_round_2 = pd.DataFrame()

for s in strings:
    for col in df: 
        df[col] = df[col].mask(~df[col].str.contains(s, na=False))
    updates_round_2[s] = df.ffill(axis=1).iloc[:, -1]
    df = split_unique_log_updates_call2.copy()

updates_round_2.iloc[:,0] = pd.to_datetime(updates_round_2.iloc[:,0].str.extract('(\d{1,4}-\d{1,2}-\d{1,2})')[0]).dt.date

updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '1'": "No answer/unable to leave VM/busy/disconnected"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '2'": "Left a message"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '3'": "Call back later"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '4'": "Hasn't received packet yet, call back in one week"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '5'": "Hasn't received packet and team needs to resend in the mail"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '6'": "Send link to the survey"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '7'": "Completed survey by phone"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '8'": "Patient refused"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '9'": "Deceased"}, regex=True)
updates_round_2.iloc[:,1] = updates_round_2.iloc[:,1].replace({" contact2_output = '10'": "Other Notes"}, regex=True)

updates_round_2.iloc[:,3] = updates_round_2.iloc[:,3].replace({" verbal_yn = '0'": "No"}, regex=True)
updates_round_2.iloc[:,3] = updates_round_2.iloc[:,3].replace({" verbal_yn = '1'": "Yes"}, regex=True)


# round 3
df = split_unique_log_updates_call3.copy()
strings = ['contact3_dt', 'contact3_output', 'contact3_nt', 'verbal_yn']
updates_round_3 = pd.DataFrame()

for s in strings:
    for col in df: 
        df[col] = df[col].mask(~df[col].str.contains(s, na=False))
    updates_round_3[s] = df.ffill(axis=1).iloc[:, -1]
    df = split_unique_log_updates_call3.copy()

updates_round_3.iloc[:,0] = pd.to_datetime(updates_round_3.iloc[:,0].str.extract('(\d{1,4}-\d{1,2}-\d{1,2})')[0]).dt.date

updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '1'": "No answer/unable to leave VM/busy/disconnected"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '2'": "Left a message"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '3'": "Call back later"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '4'": "Hasn't received packet yet, call back in one week"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '5'": "Hasn't received packet and team needs to resend in the mail"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '6'": "Send link to the survey"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '7'": "Completed survey by phone"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '8'": "Patient refused"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '9'": "Deceased"}, regex=True)
updates_round_3.iloc[:,1] = updates_round_3.iloc[:,1].replace({" contact3_output = '10'": "Other Notes"}, regex=True)

updates_round_3.iloc[:,3] = updates_round_3.iloc[:,3].replace({" verbal_yn = '0'": "No"}, regex=True)
updates_round_3.iloc[:,3] = updates_round_3.iloc[:,3].replace({" verbal_yn = '1'": "Yes"}, regex=True)




# create new dataframes with relevant columns for round 1 / 2 / 3 calls 

# round 1
clean_round_1 = split_unique_log_updates_call1['study_id'].copy()
clean_round_1 = pd.concat((clean_round_1, unique_log_updates_call1['Username']), axis=1)
clean_round_1 = pd.concat((clean_round_1, unique_log_updates_call1['Time / Date']), axis=1)
clean_round_1 = pd.concat((clean_round_1, updates_round_1), axis=1)
clean_round_1.columns = ['study_id', 'caller_username_1', 'call_timestamp_1', 'call_date_1', 'call_output_1', 'call_notes_1', 'call_verbal_consent_1']


# round 2 
clean_round_2 = split_unique_log_updates_call2['study_id'].copy()
clean_round_2 = pd.concat((clean_round_2, unique_log_updates_call2['Username']), axis=1)
clean_round_2 = pd.concat((clean_round_2, unique_log_updates_call2['Time / Date']), axis=1)
clean_round_2 = pd.concat((clean_round_2, updates_round_2), axis=1)
clean_round_2.columns = ['study_id', 'caller_username_2', 'call_timestamp_2', 'call_date_2', 'call_output_2', 'call_notes_2', 'call_verbal_consent_2']


# round 3
clean_round_3 = split_unique_log_updates_call3['study_id'].copy()
clean_round_3 = pd.concat((clean_round_3, unique_log_updates_call3['Username']), axis=1)
clean_round_3 = pd.concat((clean_round_3, unique_log_updates_call3['Time / Date']), axis=1)
clean_round_3 = pd.concat((clean_round_3, updates_round_3), axis=1)
clean_round_3.columns = ['study_id', 'caller_username_3', 'call_timestamp_3', 'call_date_3', 'call_output_3', 'call_notes_3', 'call_verbal_consent_3']


clean_round_1.columns = ['study_id', 
                         'baseline_caller_username_1', 
                         'baseline_call_timestamp_1', 
                         'baseline_call_date_1', 
                         'baseline_call_output_1', 
                         'baseline_call_notes_1', 
                         'baseline_call_verbal_consent_1']

clean_round_2.columns = ['study_id', 
                         'baseline_caller_username_2', 
                         'baseline_call_timestamp_2', 
                         'baseline_call_date_2', 
                         'baseline_call_output_2', 
                         'baseline_call_notes_2', 
                         'baseline_call_verbal_consent_2']

clean_round_3.columns = ['study_id', 
                         'baseline_caller_username_3', 
                         'baseline_call_timestamp_3', 
                         'baseline_call_date_3', 
                         'baseline_call_output_3', 
                         'baseline_call_notes_3', 
                         'baseline_call_verbal_consent_3']

## REDCap Log: (UCI ONLY) shift incorrect datetimes

In [7]:
# UCI-Only Correction: Timestamps created during 2/3 - 2/5 are 8 hours ahead, due to REDCap upgrade error
# incorrect updates range from to 02/03/2020 5:30pm to 02/06/2020 1:50am
# only the timestamps that come from the REDCap audit log are affected, not the "call_date" which is input by users
# store indeces of calls made between incorrect range
# change timestamps of incorrect calls to shift 8 hrs earlier
round_1_mistakes_index = clean_round_1[(clean_round_1['baseline_call_timestamp_1'] > '2020-02-03 16:30:00') &
                                       (clean_round_1['baseline_call_timestamp_1'] < '2020-02-06 02:00:00')].index.values.tolist()
clean_round_1.loc[round_1_mistakes_index, 'baseline_call_timestamp_1'] += pd.DateOffset(hours=-8)


round_2_mistakes_index = clean_round_2[(clean_round_2['baseline_call_timestamp_2'] > '2020-02-03 16:30:00') &
                                       (clean_round_2['baseline_call_timestamp_2'] < '2020-02-06 02:00:00')].index.values.tolist()
clean_round_2.loc[round_2_mistakes_index, 'baseline_call_timestamp_2'] += pd.DateOffset(hours=-8)


round_3_mistakes_index = clean_round_3[(clean_round_3['baseline_call_timestamp_3'] > '2020-02-03 16:30:00') &
                                       (clean_round_3['baseline_call_timestamp_3'] < '2020-02-06 02:00:00')].index.values.tolist()
clean_round_3.loc[round_3_mistakes_index, 'baseline_call_timestamp_3'] += pd.DateOffset(hours=-8)

## REDCap Log: (UCI ONLY) study ID adjustments

In [8]:
# UCI-Only Correction: One of the patients in our cohort had their MRN changed because of a mistake by the medical records
# department. This patient was intially assigned study ID xxxxxxxx2. When the patient's MRN changed, the patient was identified
# again (because they still met the Serious Illness Criteria) and assigned a new study ID xxxxxxxx9. Rick removed the first 
# record in the cohort table. However, REDCap still has the patient listed as xxxxxxxx2. I'm changing this to the updated
# study ID so the demographic data from the EMR can still be added. 
clean_round_1.loc[clean_round_1['study_id'] == UCI_ONLY_OLD_ID, 'study_id'] = UCI_ONLY_NEW_ID
clean_round_2.loc[clean_round_2['study_id'] == UCI_ONLY_OLD_ID, 'study_id'] = UCI_ONLY_NEW_ID
clean_round_3.loc[clean_round_3['study_id'] == UCI_ONLY_OLD_ID, 'study_id'] = UCI_ONLY_NEW_ID


"""
# UCI-Only Correction: One of the patients in our cohort was opted out by a provider after the surveys were mailed out. I am
# removing this patient because no calls and further contact was made. 
clean_round_1.drop(clean_round_1.loc[clean_round_1['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)
clean_round_2.drop(clean_round_2.loc[clean_round_2['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)
clean_round_3.drop(clean_round_3.loc[clean_round_3['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)
"""

"\n# UCI-Only Correction: One of the patients in our cohort was opted out by a provider after the surveys were mailed out. I am\n# removing this patient because no calls and further contact was made. \nclean_round_1.drop(clean_round_1.loc[clean_round_1['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)\nclean_round_2.drop(clean_round_2.loc[clean_round_2['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)\nclean_round_3.drop(clean_round_3.loc[clean_round_3['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)\n"

## REDCap Report: combine baseline and 12 month fields

In [9]:
# NOTE: REDCap report writes 12 month data in new rows. These need to be merged back with the baseline data. 

# split baseline and 12 month data into 2 seperate dataframes
baseline_redcap_extract = redcap_extract[redcap_extract['redcap_event_name'] == "event_baseline_arm_1"].copy()
twelve_month_redcap_extract = redcap_extract[redcap_extract['redcap_event_name'] == "event_12mo_arm_1"].copy()

# remove the empty 12 month columns from the baseline dataframe
baseline_redcap_extract = baseline_redcap_extract.dropna(axis=1, how='all')

# remove the empty baseline columns from the 12 month dataframe
twelve_month_redcap_extract = twelve_month_redcap_extract.dropna(axis=1, how='all')

# before merging, remove unnecessary 'redcap_event_name' from 12 month dataframe
twelve_month_redcap_extract = twelve_month_redcap_extract.drop('redcap_event_name', axis=1)

# UCI ONLY: remove patients who were accidentally called even though they were not mailed a 12 month survey
twelve_month_redcap_extract = twelve_month_redcap_extract.dropna(subset=['survey_mail_dt_v12'], axis=0, how='any')


# left merge 12 month data into baseline data, overwrite redcap_extract
redcap_extract = pd.merge(baseline_redcap_extract, twelve_month_redcap_extract, how='left', on='study_id')

## REDCap Report: convert number coding to strings

In [10]:
print(redcap_extract.dtypes)

study_id                             int64
redcap_event_name                   object
survey_language_sent               float64
hipaa_sent_yn                      float64
survey_completed_dt                 object
consent_received_dt                 object
survery_completed_method           float64
hipaa_received_dt                   object
caregiver_name                      object
consent_mailed_dt                   object
gift_card_type                     float64
opt_out_pat_dt                      object
opted_out_patient_reasons          float64
opted_out_patient_other             object
opted_out_patient_transcription     object
research_cohort_yn                 float64
survey_mail_dt_v12                  object
survey2_mail_dt_v132                object
survey_completed_dt_v12             object
survey_received_v12                 object
survery_completed_method_v12       float64
dtype: object


In [11]:
# NOTE: for some reason, some columns are stored as floats, which is problematic when converting to strings 

# convert the float columns to Int64 (which allows null values to coexist with ints)
redcap_extract['survey_language_sent'] = redcap_extract['survey_language_sent'].astype('Int64')
redcap_extract['hipaa_sent_yn'] = redcap_extract['hipaa_sent_yn'].astype('Int64')
redcap_extract['survery_completed_method'] = redcap_extract['survery_completed_method'].astype('Int64')
redcap_extract['gift_card_type'] = redcap_extract['gift_card_type'].astype('Int64')
redcap_extract['opted_out_patient_reasons'] = redcap_extract['opted_out_patient_reasons'].astype('Int64')
redcap_extract['research_cohort_yn'] = redcap_extract['research_cohort_yn'].astype('Int64')
redcap_extract['survery_completed_method_v12'] = redcap_extract['survery_completed_method_v12'].astype('Int64')

# convert entire dataframe to strings
redcap_extract = redcap_extract.applymap(str)

# convert dates back to datetime data type
redcap_extract['survey_completed_dt'] = pd.to_datetime(redcap_extract['survey_completed_dt']).dt.date
redcap_extract['consent_received_dt'] = pd.to_datetime(redcap_extract['consent_received_dt']).dt.date
redcap_extract['hipaa_received_dt'] = pd.to_datetime(redcap_extract['hipaa_received_dt']).dt.date
redcap_extract['consent_mailed_dt'] = pd.to_datetime(redcap_extract['consent_mailed_dt']).dt.date
redcap_extract['opt_out_pat_dt'] = pd.to_datetime(redcap_extract['opt_out_pat_dt']).dt.date
redcap_extract['survey_mail_dt_v12'] = pd.to_datetime(redcap_extract['survey_mail_dt_v12']).dt.date
redcap_extract['survey2_mail_dt_v132'] = pd.to_datetime(redcap_extract['survey2_mail_dt_v132']).dt.date
redcap_extract['survey_completed_dt_v12'] = pd.to_datetime(redcap_extract['survey_completed_dt_v12']).dt.date

redcap_extract.loc[redcap_extract['survey_language_sent'] == "0", 'survey_language_sent'] = "English"
redcap_extract.loc[redcap_extract['survey_language_sent'] == "1", 'survey_language_sent'] = "Spanish"
redcap_extract.loc[redcap_extract['survey_language_sent'] == "<NA>", 'survey_language_sent'] = np.nan

redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "0", 'hipaa_sent_yn'] = "No"
redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "1", 'hipaa_sent_yn'] = "Yes"
redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "<NA>", 'hipaa_sent_yn'] = np.nan

redcap_extract.loc[redcap_extract['survery_completed_method'] == "1", 'survery_completed_method'] = "Paper"
redcap_extract.loc[redcap_extract['survery_completed_method'] == "2", 'survery_completed_method'] = "Phone"
redcap_extract.loc[redcap_extract['survery_completed_method'] == "3", 'survery_completed_method'] = "Email"
redcap_extract.loc[redcap_extract['survery_completed_method'] == "<NA>", 'survery_completed_method'] = np.nan

redcap_extract.loc[redcap_extract['gift_card_type'] == "1", 'gift_card_type'] = "e-Gift Card"
redcap_extract.loc[redcap_extract['gift_card_type'] == "2", 'gift_card_type'] = "Physical Gift Card"
redcap_extract.loc[redcap_extract['gift_card_type'] == "3", 'gift_card_type'] = "Target Gift Card"
redcap_extract.loc[redcap_extract['gift_card_type'] == "4", 'gift_card_type'] = "Patient Declined Gift Card"
redcap_extract.loc[redcap_extract['gift_card_type'] == "<NA>", 'gift_card_type'] = np.nan

redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "1", 'opted_out_patient_reasons'] = "None Given"
redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "2", 'opted_out_patient_reasons'] = "Personal Matter"
redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "3", 'opted_out_patient_reasons'] = "Deceased"
redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "4", 'opted_out_patient_reasons'] = "Hospice"
redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "5", 'opted_out_patient_reasons'] = "Other Reason (specify)"
redcap_extract.loc[redcap_extract['opted_out_patient_reasons'] == "<NA>", 'opted_out_patient_reasons'] = np.nan

redcap_extract.loc[redcap_extract['research_cohort_yn'] == '1', 'research_cohort_yn'] = "Yes"
redcap_extract.loc[redcap_extract['research_cohort_yn'] == "<NA>", 'research_cohort_yn'] = np.nan

redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "0", 'hipaa_sent_yn'] = "No"
redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "1", 'hipaa_sent_yn'] = "Yes"
redcap_extract.loc[redcap_extract['hipaa_sent_yn'] == "<NA>", 'hipaa_sent_yn'] = np.nan

redcap_extract.loc[redcap_extract['survey_received_v12'] == "0", 'survey_received_v12'] = "No"
redcap_extract.loc[redcap_extract['survey_received_v12'] == "1", 'survey_received_v12'] = "Yes"
redcap_extract.loc[redcap_extract['survey_received_v12'] == "nan", 'survey_received_v12'] = np.nan

redcap_extract.loc[redcap_extract['survery_completed_method_v12'] == "1", 'survery_completed_method_v12'] = "Paper"
redcap_extract.loc[redcap_extract['survery_completed_method_v12'] == "2", 'survery_completed_method_v12'] = "Phone"
redcap_extract.loc[redcap_extract['survery_completed_method_v12'] == "3", 'survery_completed_method_v12'] = "Email"
redcap_extract.loc[redcap_extract['survery_completed_method_v12'] == "<NA>", 'survery_completed_method_v12'] = np.nan


# UCI ONLY: add reasons for final research cohort exclusion
redcap_extract.loc[redcap_extract['study_id'].isin(UCI_ONLY_RES_EXC_DIED), 'research_cohort_opt_out_reason'] = "Died Before Orange Dot"
redcap_extract.loc[redcap_extract['study_id'].isin(UCI_ONLY_RES_EXC_ACP), 'research_cohort_opt_out_reason'] = "Completed ACP Before Orange Dot"
redcap_extract.loc[redcap_extract['study_id'].isin(UCI_ONLY_RES_EXC_LATE_SURVEY), 'research_cohort_opt_out_reason'] = "Completed Survey After Their Intervention"
redcap_extract.loc[redcap_extract['study_id'].isin(UCI_ONLY_RES_EXC_LATE_RECEIPT), 'research_cohort_opt_out_reason'] = "Late Discovery By Study Team"



# add date that 1st bulk mailing was dropped off at USPS
redcap_extract['survey_mailing_date_1'] = pd.to_datetime('11/13/2019')
redcap_extract['survey_mailing_date_1'] = redcap_extract['survey_mailing_date_1'].dt.date

# NOTE: may want to add some additional items from Internal REDCap
useful_redcap_cols = ['study_id', 
                      'survey_language_sent',
                      'opted_out_patient_reasons',
                      'hipaa_sent_yn', 
                      'survey_completed_dt', 
                      'consent_received_dt', 
                      'survery_completed_method',  
                      'hipaa_received_dt', 
                      'consent_mailed_dt',  
                      'opt_out_pat_dt',
                      'survey_mailing_date_1', 
                      'survey_mail_dt_v12',
                      'survey2_mail_dt_v132',
                      'survey_received_v12',
                      'survery_completed_method_v12',
                      'research_cohort_opt_out_reason']



clean_redcap_extract = redcap_extract.filter(useful_redcap_cols).copy()

clean_redcap_extract.columns = ['study_id', 
                                'baseline_survey_language_sent', 
                                'patient_opt_out_reason',
                                'baseline_survey_hipaa_sent_yn', 
                                'baseline_survey_completed_date', 
                                'baseline_survey_received_date',       # rename from "consent" to "survey received" for clarity
                                'baseline_survey_completed_method',  
                                'hipaa_received_date',                 # rename to generic HIPAA received date, because it could be received at baseline or 12 months
                                'baseline_consent_mailed_date',  
                                'patient_opt_out_date',                # rename to generic patient opt out date, because opt out could happen at any time
                                'baseline_survey_mailing_date_1',
                                '12_month_survey_mailing_date_1',
                                '12_month_survey_mailing_date_2',
                                '12_month_survey_received_date',
                                '12_month_survey_completed_method',
                                'research_cohort_opt_out_reason'
                                ]

## REDCap Report: (UCI ONLY) study ID adjustments

In [12]:
# UCI-Only Correction: One of the patients in our cohort had their MRN changed because of a mistake by the medical records
# department. This patient was intially assigned study ID xxxxxxxx2. When the patient's MRN changed, the patient was identified
# again (because they still met the Serious Illness Criteria) and assigned a new study ID xxxxxxxx9. Rick removed the first 
# record in the cohort table. However, REDCap still has the patient listed as xxxxxxxx2. I'm changing this to the updated
# study ID so the demographic data from the EMR can still be added. 
clean_redcap_extract.loc[clean_redcap_extract['study_id'] == UCI_ONLY_OLD_ID, 'study_id'] = UCI_ONLY_NEW_ID

# UCI-Only Correction: One of the patients in our cohort was opted out by a provider after the surveys were mailed out. I am
# removing this patient because no calls and further contact was made. 
clean_redcap_extract.drop(clean_redcap_extract.loc[clean_redcap_extract['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)

## Baseline Survey Round 2 Mailing Lists: merge lists

In [13]:
# take the 4 mailing lists sent to mail vendor and combine 
all_mailings = [round_2_mailing_a, round_2_mailing_b, round_2_mailing_c, round_2_mailing_d]
all_round_2_mailings = pd.concat(all_mailings)

# add date that 2nd bulk mailing was dropped off at USPS 
all_round_2_mailings['baseline_survey_mailing_date_2'] = pd.to_datetime('2/18/2020')
all_round_2_mailings['baseline_survey_mailing_date_2'] = all_round_2_mailings['baseline_survey_mailing_date_2'].dt.date

# only take necessary columns
useful_round_2_mailing_cols = ['study_id', 'baseline_survey_mailing_date_2']
clean_round_2_mailings = all_round_2_mailings.filter(useful_round_2_mailing_cols).copy()

# convert study_id to string
clean_round_2_mailings = clean_round_2_mailings.astype({'study_id': str})

## Baseline Survey Round 2 Mailing Lists: (UCI ONLY) study ID adjustments

In [14]:
# UCI-Only Correction: One of the patients in our cohort had their MRN changed because of a mistake by the medical records
# department. This patient was intially assigned study ID xxxxxxxx2. When the patient's MRN changed, the patient was identified
# again (because they still met the Serious Illness Criteria) and assigned a new study ID xxxxxxxx9. Rick removed the first 
# record in the cohort table. However, REDCap still has the patient listed as xxxxxxxx2. I'm changing this to the updated
# study ID so the demographic data from the EMR can still be added. 
clean_round_2_mailings.loc[clean_round_2_mailings['study_id'] == UCI_ONLY_OLD_ID, 'study_id'] = UCI_ONLY_NEW_ID

# UCI-Only Correction: One of the patients in our cohort was opted out by a provider after the surveys were mailed out. I am
# removing this patient because no calls and further contact was made. 
clean_round_2_mailings.drop(clean_round_2_mailings.loc[clean_round_2_mailings['study_id'] == UCI_ONLY_OPTED_OUT_ID].index, inplace=True)

## Clarity Extract: fill in missing patient data

In [15]:
# UCI-Only Correction: for some reason, some patients no longer have an active MRN or their MRN changed so demographic
#    data was deleted from recent extracts


# Replace missing data with data from old extracts
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'birth_date'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 1]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'sex'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 2]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'ethnicity'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 3]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'race'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 4]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'spoken_language'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 5]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'mct_activation'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 6]
clarity_extract.loc[clarity_extract['study_id'] == UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 0], 'mct_status'] = UCI_ONLY_MISSING_PATIENT_DATA.iloc[0, 7]

## Clarity Extract: calculate Orange Dot status, age, death, etc.

In [16]:
# convert dates to datetime data type
clarity_extract['birth_date'] = pd.to_datetime(clarity_extract['birth_date'])
clarity_extract['death_date'] = pd.to_datetime(clarity_extract['death_date'])
clarity_extract['enroll_date'] = pd.to_datetime(clarity_extract['enroll_date']).dt.date
clarity_extract['date_orange'] = pd.to_datetime(clarity_extract['date_orange']).dt.date
clarity_extract['last_ad_dt'] = pd.to_datetime(clarity_extract['last_ad_dt']).dt.date
clarity_extract['letter_view'] = pd.to_datetime(clarity_extract['letter_view']).dt.date
clarity_extract['letter_view2'] = pd.to_datetime(clarity_extract['letter_view']).dt.date
clarity_extract['letter_view3'] = pd.to_datetime(clarity_extract['letter_view']).dt.date
clarity_extract['letter_view4'] = pd.to_datetime(clarity_extract['letter_view']).dt.date

# convert int to string for consistency
clarity_extract['study_id'] = clarity_extract['study_id'].astype(str)

# convert float to int for arms
clarity_extract['arm_orange'] = clarity_extract['arm_orange'].astype('Int64')
clarity_extract['arm_red'] = clarity_extract['arm_red'].astype('Int64')
clarity_extract['arm_red2'] = clarity_extract['arm_red2'].astype('Int64')
clarity_extract['arm_red3'] = clarity_extract['arm_red3'].astype('Int64')
clarity_extract['arm_red4'] = clarity_extract['arm_red4'].astype('Int64')

# assign "yes" to Orange Dot if the patient has an orange dot date, leave null for the rest of pts
clarity_extract.loc[clarity_extract['date_orange'].notnull(), 'orange_dot_yn'] = "Yes"

# assign "yes" to if patient opened their red dot #1-4 MyChart letters, leave null for the rest of pts
clarity_extract.loc[clarity_extract['letter_view'].notnull(), 'letter_view1_yn'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view2'].notnull(), 'letter_view2_yn'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view3'].notnull(), 'letter_view3_yn'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view4'].notnull(), 'letter_view4_yn'] = "Yes"

# assign "yes" to if patient opened their red dot #1-4 MyChart letters by the 24 month mark, leave null for the rest of pts
clarity_extract.loc[clarity_extract['letter_view'] < pd.to_datetime('4/12/22'), 'letter_view1_yn_24_month'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view2'] < pd.to_datetime('4/12/22'), 'letter_view2_yn_24_month'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view3'] < pd.to_datetime('4/12/22'), 'letter_view3_yn_24_month'] = "Yes"
clarity_extract.loc[clarity_extract['letter_view4'] < pd.to_datetime('4/12/22'), 'letter_view4_yn_24_month'] = "Yes"

# assign "yes" to if patient received Health Coach outreach for red dot #1-4, leave null for the rest of pts
# NOTE: This variable contains responses from the UCI Health Coaches to one of the questions in the Smart Form.
#       This represents a valid outreach attempt by the UCI Health Coaches to the patient. 
clarity_extract.loc[clarity_extract['ad_under_3yrs'].notnull(), 'arm3_outreach1_yn'] = "Yes"
clarity_extract.loc[clarity_extract['ad_under_3yrs2'].notnull(), 'arm3_outreach2_yn'] = "Yes"
clarity_extract.loc[clarity_extract['ad_under_3yrs3'].notnull(), 'arm3_outreach3_yn'] = "Yes"
clarity_extract.loc[clarity_extract['ad_under_3yrs4'].notnull(), 'arm3_outreach4_yn'] = "Yes"


# insert column of recent ACP dates from the CSV snapshot at the end of the intervention (Orange Dot + 24 months)
# NOTE: This is to account for ACP documents that may be scanned in after the intervention, but entered with an earlier date.
static_24_month_clarity_extract = static_24_month_clarity_extract[['study_id', 'last_ad_dt']]
static_24_month_clarity_extract['study_id'] = static_24_month_clarity_extract['study_id'].astype(str)
static_24_month_clarity_extract['last_ad_dt'] = pd.to_datetime(static_24_month_clarity_extract['last_ad_dt']).dt.date
static_24_month_clarity_extract.columns = ['study_id', 'last_ad_dt_24_month']
clarity_extract = pd.merge(clarity_extract, static_24_month_clarity_extract, how='outer', on='study_id')


# calculate age from today's date, unless patient is dead (then use deceased date) 
def calculate_age_current_or_death(date_of_birth, date_of_death): 
    today = date.today()
    if date_of_death is pd.NaT: 
        return today.year - date_of_birth.year
    else: 
        return date_of_death.year - date_of_birth.year

clarity_extract['age_current_or_death'] = clarity_extract.apply(lambda x: calculate_age_current_or_death(x['birth_date'], x['death_date']), axis=1)
clarity_extract['age_current_or_death'] = clarity_extract['age_current_or_death'].astype('Int64')


# calculate age from orange dot date or enrollment date if they were added after the orange dot date
def calculate_age_orange_dot_or_enrollment(date_of_birth, orange_dot_date, enrollment_date): 
    if orange_dot_date is pd.NaT: 
        return enrollment_date.year - date_of_birth.year
    else: 
        return orange_dot_date.year - date_of_birth.year

clarity_extract['age_orange_dot_or_enrollment'] = clarity_extract.apply(lambda x: calculate_age_orange_dot_or_enrollment(x['birth_date'], x['date_orange'], x['enroll_date']), axis=1)
clarity_extract['age_orange_dot_or_enrollment'] = clarity_extract['age_orange_dot_or_enrollment'].astype('Int64')


# calculate age from survey mailing date 
baseline_survey_mail_date = pd.to_datetime('11/13/2019')
def calculate_age_baseline_survey(study_id, date_of_birth): 
    if clean_redcap_extract['study_id'].str.contains(study_id).any():
        return baseline_survey_mail_date.year - date_of_birth.year

clarity_extract['age_baseline_survey'] = clarity_extract.apply(lambda x: calculate_age_baseline_survey(x['study_id'], x['birth_date']), axis=1)
clarity_extract['age_baseline_survey'] = clarity_extract['age_baseline_survey'].astype('Int64')


# aggregate all patients over the age of 89 into a group with age 90
clarity_extract.loc[clarity_extract['age_current_or_death'] > 89, 'age_current_or_death'] = 90
clarity_extract.loc[clarity_extract['age_orange_dot_or_enrollment'] > 89, 'age_orange_dot_or_enrollment'] = 90
clarity_extract.loc[clarity_extract['age_baseline_survey'] > 89, 'age_baseline_survey'] = 90


# determine if patient was deceased within 3 months of survey mailing date
# NOTE: this will include patients who died before the survey mailing, but were not known at that time
# TODO: add this for only patients who completed the baseline survey 
def determine_deceased_after_3mo_survey(study_id, date_of_death): 
    if clean_redcap_extract['study_id'].str.contains(study_id).any():
        if date_of_death < (baseline_survey_mail_date + relativedelta(months=+3)):
            return "Yes"

clarity_extract['deceased_after_3mo_of_baseline_survey'] = clarity_extract.apply(lambda x: determine_deceased_after_3mo_survey(x['study_id'], x['death_date']), axis=1)

# determine if patient is currently deceased
clarity_extract.loc[clarity_extract['death_date'].notnull(), 'current_deceased_yn'] = "Yes"


# determine if patient was deceased by the mail out of the 12 month survey 
# TODO: add this for only patients who completed the baselin survey 
clarity_extract.loc[clarity_extract['death_date'] < pd.to_datetime('4/12/21'), 'deceased_12_mo'] = "Yes"


# determine if Orange Dot patients had ACP within 3 years of the OD date
def determine_recent_ACP_for_OD(study_id, orange_dot_yn, date_orange, last_ad_dt): 
    if orange_dot_yn == "Yes":
        if (last_ad_dt > (date_orange + relativedelta(years=-3))) & (last_ad_dt < date_orange):
            return "Yes"

# get current recent ACP at OD
clarity_extract['recent_ACP_at_OD'] = clarity_extract.apply(lambda x: determine_recent_ACP_for_OD(x['study_id'], x['orange_dot_yn'], x['date_orange'], x['last_ad_dt']), axis=1)

# get 24 month static recent ACP at OD 
clarity_extract['recent_ACP_at_OD_static_24_month'] = clarity_extract.apply(lambda x: determine_recent_ACP_for_OD(x['study_id'], x['orange_dot_yn'], x['date_orange'], x['last_ad_dt_24_month']), axis=1)
        

# for consistency, make all "no" values --> null values
clarity_extract.loc[clarity_extract['research_cohort_yn'] == "Y", 'research_cohort_yn'] = "Yes"
clarity_extract.loc[clarity_extract['research_cohort_yn'] == "N", 'research_cohort_yn'] = np.NaN

clarity_extract.loc[clarity_extract['adv_cancer'] == "Y", 'adv_cancer'] = "Yes"
clarity_extract.loc[clarity_extract['adv_cancer'] == "N", 'adv_cancer'] = np.NaN

clarity_extract.loc[clarity_extract['esld'] == "Y", 'esld'] = "Yes"
clarity_extract.loc[clarity_extract['esld'] == "N", 'esld'] = np.NaN

clarity_extract.loc[clarity_extract['copd'] == "Y", 'copd'] = "Yes"
clarity_extract.loc[clarity_extract['copd'] == "N", 'copd'] = np.NaN

clarity_extract.loc[clarity_extract['chf'] == "Y", 'chf'] = "Yes"
clarity_extract.loc[clarity_extract['chf'] == "N", 'chf'] = np.NaN

clarity_extract.loc[clarity_extract['esrd'] == "Y", 'esrd'] = "Yes"
clarity_extract.loc[clarity_extract['esrd'] == "N", 'esrd'] = np.NaN

clarity_extract.loc[clarity_extract['als'] == "Y", 'als'] = "Yes"
clarity_extract.loc[clarity_extract['als'] == "N", 'als'] = np.NaN

clarity_extract.loc[clarity_extract['age_select'] == "Y", 'age_select'] = "Yes"
clarity_extract.loc[clarity_extract['age_select'] == "N", 'age_select'] = np.NaN

C:\Users\chauaj1\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\chauaj1\AppData\Local\Temp\2/ipykernel_7700/3064443333.py:116: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  if (last_ad_dt > (date_orange + relativedelta(years=-3))) & (last_ad_dt < date_orange):


## Clarity Extract: filter and format, add Current Total Cohort / Gray Dot flag

In [17]:
# NOTE: The UCI Serious Illness code was run weekly periodically starting in September 2019. The UCI intervention did not
#       go live until April 2020. During this 7 month time period, a number of the seriously ill patients expired. Therefore,
#       they were not included in the Orange Dot. Any patient that was enrolled before the Orange Dot date (4/13/2020) and 
#       was not included in the Orange Dot cohort should be excluded from the Current Total Cohort. 

# NOTE: The Orange Dot assignment happened on 4/12/2020 - 4/13/2020


### Current Total Cohort Flag ###

# make df of all patients who are NOT in the orange dot cohort
non_OD_enroll_dates = clarity_extract[['study_id', 'enroll_date']][clarity_extract['orange_dot_yn'] != "Yes"].copy()

# make df of  all patients who are not in the orange dot cohort and were enrolled before the orange dot
to_exclude = non_OD_enroll_dates[non_OD_enroll_dates['enroll_date'] < pd.to_datetime('4/12/2020')]

# flag patients as part of Current Total Cohort if they were in the Orange Dot or were enrolled after 
# stop including patients enrolled after the end of the intervention (24 months after Orange Dot date)
clarity_extract.loc[(~clarity_extract['study_id'].isin(to_exclude['study_id'])) & (clarity_extract['enroll_date'] < pd.to_datetime('4/11/22')), 'current_total_cohort_yn'] = "Yes"




### Grey Dot Flag ###

# make df of all patients who were identified before the Orange Dot date (4/12/2020)
# NOTE: at UCI, all these patients were identified on 9/17/2019
grey_dots = clarity_extract[clarity_extract['enroll_date'] < pd.to_datetime('4/12/2020')]

# flag patients as part of Grey Dot Cohort if they were enrolled before the Orange Dot
clarity_extract.loc[clarity_extract['study_id'].isin(grey_dots['study_id']), 'grey_dot_yn'] = "Yes"

# record enrollment date for Grey Dot Cohort patients
clarity_extract.loc[clarity_extract['study_id'].isin(grey_dots['study_id']), 'grey_dot_date'] = clarity_extract['enroll_date']


# select useful columns from Clarity extract
useful_clarity_cols = ['study_id', 
                       'orange_dot_yn',
                       'date_orange',
                       'enroll_date',
                       'research_cohort_yn',
                       'arm_orange',                # orange dot arm assignment 
                       'clinic_orange_name',        # orange dot clinic assignment
                       'arm',                       # current arm assignment
                       'department_name',           # current clinic assignment
                       'arm_red',                   # red dot #1 arm assignment
                       'clinic_red_name',           # red dot #1 clinic assignment 
                       'letter_view1_yn',           # red dot #1 MyChart letter opened yes/no
                       'letter_view1_yn_24_month',  # red dot #1 MyChart letter opened by 24 months yes/no
                       'arm3_outreach1_yn',         # red dot #1 Health Coach outreach yes/no
                       'arm_red2',
                       'clinic_red2_name',
                       'letter_view2_yn',
                       'letter_view2_yn_24_month',
                       'arm3_outreach2_yn',
                       'arm_red3',
                       'clinic_red3_name',
                       'letter_view3_yn',
                       'letter_view3_yn_24_month',
                       'arm3_outreach3_yn',
                       'arm_red4',
                       'clinic_red4_name',
                       'letter_view4_yn',
                       'letter_view4_yn_24_month',
                       'arm3_outreach4_yn',
                       'mct_status',            # MyChart status 
                       'age_current_or_death',
                       'age_orange_dot_or_enrollment',
                       'age_baseline_survey',
                       'deceased_after_3mo_of_baseline_survey',
                       'current_deceased_yn',
                       'sex', 
                       'race', 
                       'ethnicity',
                       'spoken_language', 
                       'written_language',
                       'insurance',            # insurance was pulled by Rick, not sure if it conforms to UCSF's categorization
                       'religion',
                       'adv_cancer',
                       'esld',
                       'copd',
                       'chf',
                       'esrd',
                       'als',
                       'age_select',
                       'current_total_cohort_yn',
                       'grey_dot_yn',
                       'grey_dot_date',
                       'last_ad_dt',
                       'death_date', 
                       'deceased_12_mo',
                       'recent_ACP_at_OD',
                       'recent_ACP_at_OD_static_24_month'
                       ]

clean_clarity_extract = clarity_extract.filter(useful_clarity_cols).copy()

# rename columns for consistency  
clean_clarity_extract.columns = ['study_id', 
                                         'orange_dot_yn',
                                         'orange_dot_date',
                                         'enroll_date',
                                         'research_cohort_yn',
                                         'arm_orange_dot',
                                         'clinic_orange_dot',
                                         'arm_current',                                         
                                         'clinic_current',
                                         'arm_red_dot_1',
                                         'clinc_red_dot_1',
                                         'mychart_opened_yn_red_dot_1', 
                                         'mychart_opened_yn_static_24_month_red_dot_1',
                                         'health_coach_outreach_yn_red_dot_1',
                                         'arm_red_dot_2',
                                         'clinc_red_dot_2',
                                         'mychart_opened_yn_red_dot_2',
                                         'mychart_opened_yn_static_24_month_red_dot_2',
                                         'health_coach_outreach_yn_red_dot_2',
                                         'arm_red_dot_3',
                                         'clinc_red_dot_3',
                                         'mychart_opened_yn_red_dot_3',
                                         'mychart_opened_yn_static_24_month_red_dot_3',
                                         'health_coach_outreach_yn_red_dot_3',
                                         'arm_red_dot_4',
                                         'clinc_red_dot_4',
                                         'mychart_opened_yn_red_dot_4',
                                         'mychart_opened_yn_static_24_month_red_dot_4',
                                         'health_coach_outreach_yn_red_dot_4',
                                         'mychart_status',
                                         'age_current_or_death',
                                         'age_orange_dot_or_enrollment',
                                         'age_baseline_survey',
                                         'deceased_after_3mo_of_baseline_survey',
                                         'deceased_current',
                                         'sex', 
                                         'race', 
                                         'ethnicity',
                                         'spoken_language', 
                                         'written_language',
                                         'insurance',
                                         'religion',
                                         'illness_adv_cancer',
                                         'illness_ESLD',
                                         'illness_COPD',
                                         'illness_CHF',
                                         'illness_ESRD',
                                         'illness_ALS',
                                         'illness_age',
                                         'current_total_cohort_yn',
                                         'grey_dot_yn',
                                         'grey_dot_date',
                                         'last_ad_dt',
                                         'death_date',
                                         'deceased_12_mo', 
                                         'recent_ACP_at_OD',
                                         'recent_ACP_at_OD_static_24_month'
                                         ]

C:\Users\chauaj1\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


## Population Cohort GeoID: merge on Census tract codes to find SVI 

In [18]:
useful_SVI_cols = ['FIPS',          # concatenation of geocoding IDs, including Census Tract code
                   'RPL_THEME1',    # Javi: percentile ranking for socioeconomic theme
                   'RPL_THEME2',    # Javi: percentile ranking for household composition theme
                   'RPL_THEME3',    # Javi: percentile ranking for minority status and language theme
                   'RPL_THEME4',    # Javi: percentile ranking for housing type and transportation composition theme
                   'RPL_THEMES']    # Javi: overall percentile ranking for SVI

clean_SVI_source = SVI_source.filter(useful_SVI_cols).copy()

clean_SVI_source.columns = ['FIPS',
                            'SVI_socioeconomic',
                            'SVI_household_comp',
                            'SVI_minority_language',
                            'SVI_housing_transportation',
                            'SVI_total']


# NOTE: for some reason, removing the 1st and last number from the Census "GEOID" value = the CDC's "FIPS" value
pop_geoID['FIPS'] = pop_geoID['GEOID'].str[1:-1]

# convert CDC's "FIPS" int value to match Census's "GEOID" string value
clean_SVI_source['FIPS'] = clean_SVI_source['FIPS'].astype(str)

pop_geoID_SVI = pd.merge(pop_geoID, clean_SVI_source, how='left', on='FIPS')

pop_geoID_SVI['study_id'] = pop_geoID_SVI['study_id'].astype(str)

useful_pop_geoID_SVI_col = ['study_id',
                            'SVI_socioeconomic',
                            'SVI_household_comp',
                            'SVI_minority_language',
                            'SVI_housing_transportation',
                            'SVI_total']

clean_pop_geoID_SVI = pop_geoID_SVI.filter(useful_pop_geoID_SVI_col).copy()

## Join REDCap Extract, REDCap Log, Geocoded Data, and Clarity Extract

In [19]:
# merge round 1 / 2 / 3 calls so there is 1 patient per line with all call data in columns 
clean_1_2_calls = pd.merge(clean_round_1, clean_round_2, how='left', on='study_id')
clean_all_calls = pd.merge(clean_1_2_calls, clean_round_3, how='left', on='study_id')

# join patients who were called with patients who returned surveys (source: REDCap report / extract)
calls_and_redcap = pd.merge(clean_all_calls, clean_redcap_extract, how='outer', on='study_id')

# add 2nd round mailing dates
calls_and_redcap_and_mailing = pd.merge(calls_and_redcap, clean_round_2_mailings, how='left', on='study_id')

# include all other pts in population cohort and add demographic data 
pop_cohort = pd.merge(calls_and_redcap_and_mailing, clean_clarity_extract, how='outer', on='study_id')

# add SVI data 
full_pop_cohort = pd.merge(pop_cohort, clean_pop_geoID_SVI, how='left', on='study_id')

## Grey Dot Exclusions

In [20]:
# UCI ONLY: add reasons for grey dot patients' inelligibility for baseline survey (PCP exclusions from emails)
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_NO_REASON), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: no reason provided"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_NO_ENG_SP), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Does Not Speak English / Spanish"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_AD_POLST), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Recent AD / POLST"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_DEMENTIA), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Dementia"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_WRONG_PCP), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Wrong PCP"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_MOVED), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Moved Out of Area"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_HOSPICE), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Hospice"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_GREY_DOT_EXC_PCP_INCAPABLE), 'grey_dot_ineligible_for_baseline_survey_reason'] = "PCP Exclusion: Incapable"

# UCI ONLY: add reasons for grey dot patients' inelligibility for baseline survey (ACP within 3 years of baseline survey mail out date)
full_pop_cohort.loc[((full_pop_cohort['grey_dot_yn'].notnull()) &
                     (full_pop_cohort['baseline_survey_mailing_date_1'].isnull()) &
                     (full_pop_cohort['last_ad_dt'] > pd.to_datetime('11/13/2016')) &
                     (full_pop_cohort['last_ad_dt'] < pd.to_datetime('11/13/2019'))), 'grey_dot_ineligible_for_baseline_survey_reason'] = "Recent ACP as of Baseline Mailing Date"

# UCI ONLY: add reasons for grey dot patients' inelligibility for baseline survey (deceased)
full_pop_cohort.loc[((full_pop_cohort['grey_dot_yn'].notnull()) & 
                     (full_pop_cohort['baseline_survey_mailing_date_1'].isnull()) &
                     (full_pop_cohort['death_date'] < pd.to_datetime('11/13/2019'))), 'grey_dot_ineligible_for_baseline_survey_reason'] = "Deceased as of Baseline Mailing Date"

# UCI ONLY: add reasons for grey dot patients' inelligibility for baseline survey (other)
full_pop_cohort.loc[((full_pop_cohort['grey_dot_yn'].notnull()) & 
                     (full_pop_cohort['baseline_survey_mailing_date_1'].isnull()) &
                     (full_pop_cohort['grey_dot_ineligible_for_baseline_survey_reason'].isnull())), 'grey_dot_ineligible_for_baseline_survey_reason'] = "Other Reason (unknown reason, clerical error, or outdated data)"

## Orange Dot Exclusions

In [21]:
# UCI ONLY: add reasons for orange dot exclusion (patients in grey dot who were deceased / on hospice)
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_ORANGE_DOT_EXC_DECEASED), 'orange_dot_exclusion_reason'] = "Study Team Reported as Deceased (REDCap)"
full_pop_cohort.loc[full_pop_cohort['study_id'].isin(UCI_ONLY_ORANGE_DOT_EXC_HOSPICE), 'orange_dot_exclusion_reason'] = "Study Team Reported as Hospice (REDCap)"


# UCI ONLY: add reasons for orange dot exclusion (patients in grey dot who were deceased by orange dot date)
full_pop_cohort.loc[((full_pop_cohort['grey_dot_yn'].notnull()) &
                     (full_pop_cohort['orange_dot_yn'].isnull()) & 
                     (full_pop_cohort['orange_dot_exclusion_reason'].isnull()) & 
                     (full_pop_cohort['death_date'] <= pd.to_datetime('4/12/2020'))), 'orange_dot_exclusion_reason'] = "Deceased as of Orange Dot Date (Epic EMR)"

# remove unncessary PHI
full_pop_cohort = full_pop_cohort.drop(['last_ad_dt', 'death_date'], axis=1)

## Reorder columns

In [22]:
full_pop_cohort = full_pop_cohort[['study_id',                                      # demographics
                                   'sex', 
                                   'race', 
                                   'ethnicity', 
                                   'spoken_language', 
                                   'age_current_or_death', 
                                   'age_orange_dot_or_enrollment', 
                                   'age_baseline_survey', 
                                   'deceased_after_3mo_of_baseline_survey',
                                   'deceased_current',
                                   'deceased_12_mo', 
                                   'insurance', 
                                   'religion', 
                                   'illness_adv_cancer',                           # serious illnesses
                                   'illness_ESLD', 
                                   'illness_COPD', 
                                   'illness_CHF', 
                                   'illness_ESRD',
                                   'illness_ALS',
                                   'illness_age',
                                   'SVI_socioeconomic',                            # geocoded SVI scores
                                   'SVI_household_comp', 
                                   'SVI_minority_language', 
                                   'SVI_housing_transportation', 
                                   'SVI_total',
                                   'research_cohort_yn',                           # overall enrollment
                                   'research_cohort_opt_out_reason',
                                   'hipaa_received_date',
                                   'patient_opt_out_date',
                                   'patient_opt_out_reason',
                                   'baseline_survey_mailing_date_1',               # baseline survey recruitment data
                                   'baseline_survey_mailing_date_2',
                                   'baseline_survey_language_sent', 
                                   'baseline_survey_hipaa_sent_yn', 
                                   'baseline_survey_completed_date', 
                                   'baseline_survey_received_date', 
                                   'baseline_survey_completed_method', 
                                   'baseline_consent_mailed_date', 
                                   'baseline_caller_username_1', 
                                   'baseline_call_timestamp_1', 
                                   'baseline_call_date_1', 
                                   'baseline_call_output_1', 
                                   'baseline_call_verbal_consent_1', 
                                   'baseline_caller_username_2', 
                                   'baseline_call_timestamp_2', 
                                   'baseline_call_date_2', 
                                   'baseline_call_output_2',  
                                   'baseline_call_verbal_consent_2', 
                                   'baseline_caller_username_3', 
                                   'baseline_call_timestamp_3', 
                                   'baseline_call_date_3', 
                                   'baseline_call_output_3', 
                                   'baseline_call_verbal_consent_3',
                                   '12_month_survey_mailing_date_1',              # 12 month survey recruitment data
                                   '12_month_survey_mailing_date_2',
                                   '12_month_survey_received_date',
                                   '12_month_survey_completed_method',
                                   'enroll_date',                                 # intervention data  
                                   'current_total_cohort_yn',
                                   'orange_dot_yn',
                                   'orange_dot_date',
                                   'orange_dot_exclusion_reason',
                                   'recent_ACP_at_OD',
                                   'recent_ACP_at_OD_static_24_month',
                                   'grey_dot_yn',
                                   'grey_dot_date',
                                   'grey_dot_ineligible_for_baseline_survey_reason',
                                   'arm_current',
                                   'clinic_current',
                                   'arm_orange_dot', 
                                   'clinic_orange_dot',
                                   'arm_red_dot_1',
                                   'clinc_red_dot_1',
                                   'mychart_opened_yn_red_dot_1', 
                                   'mychart_opened_yn_static_24_month_red_dot_1',
                                   'health_coach_outreach_yn_red_dot_1',
                                   'arm_red_dot_2',
                                   'clinc_red_dot_2',
                                   'mychart_opened_yn_red_dot_2', 
                                   'mychart_opened_yn_static_24_month_red_dot_2',
                                   'health_coach_outreach_yn_red_dot_2',
                                   'arm_red_dot_3',
                                   'clinc_red_dot_3',
                                   'mychart_opened_yn_red_dot_3', 
                                   'mychart_opened_yn_static_24_month_red_dot_3',
                                   'health_coach_outreach_yn_red_dot_3',
                                   'arm_red_dot_4',
                                   'clinc_red_dot_4',
                                   'mychart_opened_yn_red_dot_4', 
                                   'mychart_opened_yn_static_24_month_red_dot_4',
                                   'health_coach_outreach_yn_red_dot_4',
                                   'mychart_status'
                                   ]]

In [23]:
print(full_pop_cohort.dtypes)

study_id                                                  object
sex                                                       object
race                                                      object
ethnicity                                                 object
spoken_language                                           object
age_current_or_death                                       Int64
age_orange_dot_or_enrollment                               Int64
age_baseline_survey                                        Int64
deceased_after_3mo_of_baseline_survey                     object
deceased_current                                          object
deceased_12_mo                                            object
insurance                                                 object
religion                                                  object
illness_adv_cancer                                        object
illness_ESLD                                              object
illness_COPD             

In [24]:
print("Current Total Patients: ", len(full_pop_cohort))
print("Current Total Cohort: ", full_pop_cohort['current_total_cohort_yn'].count())
print("Orange Dot Cohort: ", full_pop_cohort['orange_dot_yn'].count())
print("Grey Dot Cohort: ", full_pop_cohort['grey_dot_yn'].count())
print("Research Cohort: ", full_pop_cohort['research_cohort_yn'].count())

print("\nMailed Baseline Surveys: ", full_pop_cohort['baseline_survey_mailing_date_1'].count())
print("Received Baseline Surveys: ", full_pop_cohort['baseline_survey_received_date'].count())
print("Excluded Baseline Surveys: ", full_pop_cohort['research_cohort_opt_out_reason'].count())

print("\nMailed 12 Month Surveys: ", full_pop_cohort['12_month_survey_mailing_date_1'].count())
print("Received 12 Month Surveys: ", full_pop_cohort['12_month_survey_received_date'].count())

print("\nOpted Out Since Baseline Completion: ", full_pop_cohort.loc[(full_pop_cohort['patient_opt_out_date'].notnull()) & 
                                                                     (full_pop_cohort['baseline_survey_received_date'].notnull()), 'study_id'].count())

print("Completed Baseline, Deceased at 12 Months: ", full_pop_cohort.loc[(full_pop_cohort['baseline_survey_received_date'].notnull()) & 
                                                                     (full_pop_cohort['deceased_12_mo'].notnull()), 'study_id'].count())

Current Total Patients:  2773
Current Total Cohort:  2582
Orange Dot Cohort:  1354
Grey Dot Cohort:  1152
Research Cohort:  172

Mailed Baseline Surveys:  891
Received Baseline Surveys:  182
Excluded Baseline Surveys:  10

Mailed 12 Month Surveys:  161
Received 12 Month Surveys:  140

Opted Out Since Baseline Completion:  4
Completed Baseline, Deceased at 12 Months:  18


In [25]:
full_pop_cohort.to_csv('data/processed/UCI_recruitment_cohort10.csv', index=False)

In [26]:
print("OD Patients: ", full_pop_cohort['orange_dot_yn'].value_counts())

print("OD Patients with recent ACP: ", full_pop_cohort.groupby('arm_orange_dot')['recent_ACP_at_OD'].value_counts())

print("OD Patients by arm: ", full_pop_cohort.groupby('arm_orange_dot')['orange_dot_yn'].value_counts())

OD Patients:  Yes    1354
Name: orange_dot_yn, dtype: int64
OD Patients with recent ACP:  arm_orange_dot  recent_ACP_at_OD
1               Yes                  38
2               Yes                 119
3               Yes                  22
Name: recent_ACP_at_OD, dtype: int64
OD Patients by arm:  arm_orange_dot  orange_dot_yn
1               Yes              454
2               Yes              701
3               Yes              199
Name: orange_dot_yn, dtype: int64
